In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [51]:
## read the tsv files in MovieSummaries
load_movies = pd.read_csv('MovieSummaries/movie.metadata.tsv', sep = '\t', header = None)
load_character = pd.read_csv('MovieSummaries/character.metadata.tsv', sep = '\t', header = None)



In [52]:
movies = load_movies.copy()
character = load_character.copy()


In [53]:
# Make the name of the colum of movie: wikip_ID, freebase_ID, name, release date, box office, runtime, languages, countries, genres	
movies.columns = ['Wiki_ID', 'Freebase_ID', 'name', 'release_date', 'box_office', 'runtime', 'languages', 'countries', 'genres']
# Make the name of the columns of character wiki_ID, freebase_ID, character_name, actor_DOB, actor_gender, actor_height, actor_ethnicity, actor_name, actor_age, freebase_character_map
character.columns = ['Wiki_ID', 'Freebase_ID','release_date', 'character_name', 'actor_DOB', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 'actor_age', '10', '11', '12']
character = character.drop(['10', '11', '12'], axis = 1)

In [54]:
## read grossed merged clean csv
gross = pd.read_csv('gross_merged_clean.csv')
gross

,tconst,Wiki_ID,Freebase_ID,country,movie,gross
0,tt0228333,975900,/m/03vyhn,Domestic,ghosts of mars,8709640.0
1,tt0228333,975900,/m/03vyhn,France,ghosts of mars,1937390.0
2,tt0228333,975900,/m/03vyhn,Spain,ghosts of mars,509536.0
3,tt0228333,975900,/m/03vyhn,Germany,ghosts of mars,496679.0
4,tt0228333,975900,/m/03vyhn,United Kingdom,ghosts of mars,148898.0
...,...,...,...,...,...,...
90661,tt0120202,1918494,/m/0660qx,Denmark,state and main,13732.0
90662,tt0120202,1918494,/m/0660qx,Australia,state and main,385631.0
90663,tt0107057,664006,/m/030xw6,Domestic,guilty as sin,22866222.0
90664,tt1606259,34980460,/m/0g4pl34,Domestic,knuckle,2647.0


## Question 1

For The first question we will need the gross of the movies in the different region of the world, as well as their release year and their genre.


In [55]:
# The release date of the movie is in the format of YYYY-MM-DD, we only need the year 
# so we split the date and only keep the year
movies_clean_release = movies.copy()
movies_clean_release['release_date'] = movies_clean_release['release_date'].str[:4]
movies_clean_release['release_date'] = pd.to_numeric(movies_clean_release['release_date'], errors='coerce')
# Here the errors ignore is to keep the nan in the release date column
movies_clean_release['release_date'] = movies_clean_release['release_date'].astype(np.int64, errors='ignore')

In [56]:
movies_clean_genre = movies_clean_release.copy()
movies_clean_genre['genres'] = movies_clean_genre['genres'].apply(lambda x: list(ast.literal_eval(x).values()))
# This line is to make the empty list to be nan so the nan are uniform
movies_clean_genre['genres'] = movies_clean_genre['genres'].apply(lambda x: x if len(x) != 0 else np.nan)


In [57]:
gross_genre_release = gross.copy()
#merge gross and movies_clean_genre on wiki_ID
gross_genre_release = gross_genre_release.merge(movies_clean_genre[['Wiki_ID', 'release_date', 'genres']], on = 'Wiki_ID', how = 'left')


In [58]:
# count the numers of nan in genre
missing_genre_value = gross_genre_release.drop_duplicates(subset='Wiki_ID')['genres'].isna().sum()/gross_genre_release.drop_duplicates(subset='Wiki_ID')['genres'].shape[0]*100
print('The percentage of missing values in the genre column is {:.2f}%'.format(missing_genre_value))



The percentage of missing values in the genre column is 0.14%


We have a very low percentage of genre missing we can therefore have a good analysis about this metric. We don't have any missing values for the release date because it was used as a key when merging the IMdB dataset with the CMU datset

## Question 3 

In [59]:
movies_clean_country = movies_clean_genre.copy()
movies_clean_country['countries'] = movies_clean_country['countries'].apply(lambda x: list(ast.literal_eval(x).values()))
# This line is to make the empty list to be nan so the nan are uniform
movies_clean_country['countries'] = movies_clean_country['countries'].apply(lambda x: x if len(x) != 0 else np.nan)

In [60]:
gross_country = gross.copy()
#merge gross and movies_clean_genre on wiki_ID
gross_country = gross_country.merge(movies_clean_country[['Wiki_ID', 'countries']], on = 'Wiki_ID', how = 'left')

In [61]:
# count the numers of nan in countries
missing_country_value = gross_country.drop_duplicates(subset='Wiki_ID')['countries'].isna().sum()/gross_country.drop_duplicates(subset='Wiki_ID')['countries'].shape[0]*100
print('The percentage of missing values in the countries column is {:.2f}%'.format(missing_country_value))

The percentage of missing values in the countries column is 1.70%


In [62]:
coprood_count = gross_country.dropna(subset='countries')
coprood_count = coprood_count.drop_duplicates(subset='Wiki_ID')[coprood_count['countries'].apply(lambda x: len(x)) > 1].shape[0]/gross_country.drop_duplicates(subset='Wiki_ID')['countries'].shape[0]*100

print('The number of movies that are co-produced by more than one country is {:.2f}%'.format(coprood_count))

The number of movies that are co-produced by more than one country is 24.98%


C:\Users\marcp\AppData\Local\Temp\ipykernel_9812\30425266.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  coprood_count = coprood_count.drop_duplicates(subset='Wiki_ID')[coprood_count['countries'].apply(lambda x: len(x)) > 1].shape[0]/gross_country.drop_duplicates(subset='Wiki_ID')['countries'].shape[0]*100


We have a very low percentage of countries missing we can therefore have a good analysis about this metric. And the level of co-production is relatively high so it will still make sense to do our analysis.

## Question 4

For the gender representation in the moviee. We decided to only use the values that were given for each of the movies, making the assumption that if the gender of an actor is not given the character was probably a small part. 

In [63]:
# go in the character dataset and find the percentage of gender F in each movie
character_gender = character.copy()

#count the number of "F" for each wiki_ID in character_gender
# groupby wiki_ID and count the number of "F" in each group
character_groupby = character_gender.groupby('Wiki_ID')
character_groupby_tot = character_groupby["actor_gender"].count()
character_groupby_F = character_groupby.apply(lambda x: (x['actor_gender'] == 'F').sum())
ratio_female_to_total = character_groupby_F / character_groupby_tot
print(ratio_female_to_total)






Wiki_ID
330         1.000000
3217        0.230769
3333        0.428571
3746        0.214286
3837        0.100000
              ...   
37373877    0.500000
37476824    0.400000
37478048    0.250000
37492363         NaN
37501922    0.500000
Length: 64330, dtype: float64


In [67]:
character[character['Wiki_ID']==37501922    ]

,Wiki_ID,Freebase_ID,release_date,character_name,actor_DOB,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age
429940,37501922,/m/0c0m5vt,1992,John Hunter,1966-10-11,M,1.765,NaN,Luke Perry,25.0
429941,37501922,/m/0c0m5vt,1992,Craig Murphy,1969-07-28,F,1.720,/m/041rx,Alexis Arquette,22.0
